<img src="https://images.unsplash.com/photo-1549517045-bc93de075e53?ixid=MnwxMjA3fDB8MHxzZWFyY2h8MTh8fGhvdXNlfGVufDB8fDB8fA%3D%3D&ixlib=rb-1.2.1&w=1000&q=80" style="float: left; margin: 20px; height: 55px">


# Introduction

<font size = 3> I am part of the City of Ames Housing division under the mayor-council government. As part of our growing population mission; providing decent, safe, affordable housing while enchancing the quality of life for econmically challenged inviduals. <br><br>  As there are factors that contribute to the saleprice of houses, we will be exploring these factors through past dataset of house saleprices collated from 2006 to 2010. We believe that as we identify the more important features, we will be able to further aid economically challenged indviduals in choosing the best house and its respective features best suited for their needs <br><br> 

<font size = 4> **Problem statement**

<font size = 3>What are the features that strongly attributes to saleprice of houses?
    
<font size = 3>A model will be built to determine these features, the metrics that will be optimised and how these metrics will be used to measure success

# Imports

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso,LassoCV, ElasticNet, ElasticNetCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error

<font size = 4> **Datasets used**

* [`new_test.csv`](./datasets/new_test.csv): test dataset to be cleaned
* [`train.csv`](./datasets/train.csv): train dataset to to be cleaned


In [2]:
test = pd.read_csv('datasets/new_test.csv')
train = pd.read_csv('datasets/train.csv')

# Data cleaning and imputing

Converting dataframe to all lowercase for Train and Test

In [3]:
train.columns = [c.lower().replace(' ', '_') for c in train.columns]
train.head(2) #check if columns in train has been replaced 

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,saleprice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000


In [4]:
test.columns = [c.lower().replace(' ', '_') for c in test.columns]
test.head(2) #check if columns in test has been replaced 

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,...,0,0,0,NaN,NaN,NaN,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NaN,IR1,Lvl,...,0,0,0,NaN,NaN,NaN,0,8,2006,WD


## datatype check

check for train datatype

In [5]:
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2051 entries, 0 to 2050
Data columns (total 81 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2051 non-null   int64  
 1   pid              2051 non-null   int64  
 2   ms_subclass      2051 non-null   int64  
 3   ms_zoning        2051 non-null   object 
 4   lot_frontage     1721 non-null   float64
 5   lot_area         2051 non-null   int64  
 6   street           2051 non-null   object 
 7   alley            140 non-null    object 
 8   lot_shape        2051 non-null   object 
 9   land_contour     2051 non-null   object 
 10  utilities        2051 non-null   object 
 11  lot_config       2051 non-null   object 
 12  land_slope       2051 non-null   object 
 13  neighborhood     2051 non-null   object 
 14  condition_1      2051 non-null   object 
 15  condition_2      2051 non-null   object 
 16  bldg_type        2051 non-null   object 
 17  house_style   

converting train datatype to float

In [6]:
train['garage_yr_blt'] = train['garage_yr_blt'].map(lambda x: float(x))

In [7]:
train['garage_yr_blt'].dtype

dtype('float64')

check for test datatype

In [8]:
test.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               879 non-null    int64  
 1   pid              879 non-null    int64  
 2   ms_subclass      879 non-null    int64  
 3   ms_zoning        879 non-null    object 
 4   lot_frontage     719 non-null    float64
 5   lot_area         879 non-null    int64  
 6   street           879 non-null    object 
 7   alley            58 non-null     object 
 8   lot_shape        879 non-null    object 
 9   land_contour     879 non-null    object 
 10  utilities        879 non-null    object 
 11  lot_config       879 non-null    object 
 12  land_slope       879 non-null    object 
 13  neighborhood     879 non-null    object 
 14  condition_1      879 non-null    object 
 15  condition_2      879 non-null    object 
 16  bldg_type        879 non-null    object 
 17  house_style     

## Removal of columns not affecting saleprice in test and train

Check columns for Train

In [9]:
len(train.columns)

81

In [10]:
train.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

Check columns for Test

In [11]:
len(test.columns)

80

In [12]:
test.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'alley', 'lot_shape', 'land_contour', 'utilities',
       'lot_config', 'land_slope', 'neighborhood', 'condition_1',
       'condition_2', 'bldg_type', 'house_style', 'overall_qual',
       'overall_cond', 'year_built', 'year_remod/add', 'roof_style',
       'roof_matl', 'exterior_1st', 'exterior_2nd', 'mas_vnr_type',
       'mas_vnr_area', 'exter_qual', 'exter_cond', 'foundation', 'bsmt_qual',
       'bsmt_cond', 'bsmt_exposure', 'bsmtfin_type_1', 'bsmtfin_sf_1',
       'bsmtfin_type_2', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf',
       'heating', 'heating_qc', 'central_air', 'electrical', '1st_flr_sf',
       '2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'bsmt_full_bath',
       'bsmt_half_bath', 'full_bath', 'half_bath', 'bedroom_abvgr',
       'kitchen_abvgr', 'kitchen_qual', 'totrms_abvgrd', 'functional',
       'fireplaces', 'fireplace_qu', 'garage_type', 'garage_yr_blt',
       'g

Check if id is repeated in test and train

In [13]:
train['id'].isin(test['id']).value_counts()

False    2051
Name: id, dtype: int64

In [14]:
train['pid'].isin(test['pid']).value_counts()

False    2051
Name: pid, dtype: int64

Both columns; PID and ID, are identification numbers and are not repeated in both trainset and will be dropped.
Sale Type column will also be dropped due to it being a feature acquired after sale.

Drop for train

In [15]:
train.drop(columns =['id', 'pid', 'sale_type'], inplace=True)

Check for number of columns left

In [16]:
len(train.columns)

78

Drop for test

In [17]:
test.drop(columns = ['id', 'pid'], inplace=True)

Check for number of columns left in test

In [18]:
len(train.columns)

78

## Null value(s) check and imputing for train dataset and test correspondingly

In [19]:
train[train.columns[train.isnull().any()]].isnull().sum().sort_values(ascending=False)

pool_qc           2042
misc_feature      1986
alley             1911
fence             1651
fireplace_qu      1000
lot_frontage       330
garage_yr_blt      114
garage_cond        114
garage_qual        114
garage_finish      114
garage_type        113
bsmt_exposure       58
bsmtfin_type_2      56
bsmt_cond           55
bsmt_qual           55
bsmtfin_type_1      55
mas_vnr_area        22
mas_vnr_type        22
bsmt_half_bath       2
bsmt_full_bath       2
total_bsmt_sf        1
bsmt_unf_sf          1
bsmtfin_sf_2         1
garage_cars          1
garage_area          1
bsmtfin_sf_1         1
dtype: int64

### pool_qc 

There are 2 columns related to pool, pool_area and pool_qc<br>
pool_qc can only be NA = No Pool, when pool_area is 0(indicator no pool)

In [20]:
train.loc[(train['pool_area']>0) & (train['pool_qc'].isnull()), ['pool_area', 'pool_qc']]
#check for columns where pool_qc is nan for pool_area is more than 0

,pool_area,pool_qc


Since all pool_qc NAN values are rows where pool area is zero, we can assign all nan values to be NA(no pool)

In [21]:
train['pool_qc'].fillna(value = 'NA', inplace = True)

In [22]:
test['pool_qc'].fillna(value = 'NA', inplace = True)

### misc_feature

misc feature has relation to misc_val,if misc_val is not 0, musc_feature null values cannot be NA(none).<br>


In [23]:
train.loc[(train['misc_feature'].isnull())&(train['misc_val']!=0),['misc_feature', 'misc_val']]

,misc_feature,misc_val


Since all null values of misc_feature is under misc_val value of 0, all nan values of misc_feature will be replaced by 'NA'

In [24]:
train['misc_feature'].fillna(value = 'NA', inplace = True)

In [25]:
test['misc_feature'].fillna(value = 'NA', inplace = True)

### alley

There is no deciding feature that determines the type of alley access to property, all values will be replaced as NA(no alley access)

In [26]:
train['alley'].fillna(value = 'NA', inplace=True)

In [27]:
test['alley'].fillna(value = 'NA', inplace=True)

In [28]:
train['alley'].unique()

array(['NA', 'Pave', 'Grvl'], dtype=object)

There is no deciding feature that determines the type of fence, all values will be replaced as NA(no fence)

In [29]:
train['fence'].fillna(value = 'NA', inplace = True)

In [30]:
test['fence'].fillna(value = 'NA', inplace = True)

### fireplace_qu

fireplace_qu is related to fireplaces, fireplace_qu cannot be null if number of fireplaces > 0 <br>
check which fireplace_qu rowis null, when fireplaces is more than 0

In [31]:
train.loc[(train['fireplaces']>0
           ) & (train['fireplace_qu'].isnull()), ['fireplaces', 'fireplace_qu']]

,fireplaces,fireplace_qu


since there is no row with fireplace_qu null values, when fireplaces is more than 0<br>
all fireplace_qu null values will be replaced as no fireplace

In [32]:
train['fireplace_qu'].fillna(value = 'NA', inplace=True)

In [33]:
test['fireplace_qu'].fillna(value = 'NA', inplace=True)

### lot_frontage

finding lot shape of all respective null values

In [34]:
train.loc[train['lot_frontage'].isnull()==True,'lot_shape'].unique()

array(['IR1', 'Reg', 'IR2', 'IR3'], dtype=object)

#### Check relationship of lot_area to lot_frontage, for the same lot_shape.
#### and apply respective porportionality if present

##### lot_shape = Reg

In [35]:
train.loc[((train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna())
          ),['lot_shape','lot_area','lot_frontage']
         ].sort_values(by='lot_area',ascending=False)

,lot_shape,lot_area,lot_frontage
1875,Reg,43500,200.0
24,Reg,34650,100.0
635,Reg,31250,125.0
12,Reg,26400,120.0
1432,Reg,25000,100.0
...,...,...,...
1447,Reg,1484,21.0
787,Reg,1477,21.0
53,Reg,1476,21.0
1544,Reg,1470,21.0


sorting unique values of regular shape for cross reference to lot_area

In [36]:
np.sort(train.loc[((train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna())
          ),['lot_shape','lot_area','lot_frontage']
         ]['lot_frontage'].unique())

array([ 21.,  24.,  30.,  32.,  33.,  34.,  35.,  36.,  37.,  40.,  41.,
        42.,  43.,  44.,  45.,  46.,  47.,  48.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,
        87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  98.,
        99., 100., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 119., 120., 125., 128., 129., 130.,
       134., 140., 144., 150., 160., 200.])

Replacement of null lot_frontage for lot_area <10000, with median value of lot_frontage for area < 10000

In [37]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].isnull()
             ),['lot_frontage','lot_area','lot_shape']]

,lot_frontage,lot_area,lot_shape
8,NaN,15783,Reg
50,NaN,2998,Reg
76,NaN,17529,Reg
106,NaN,2651,Reg
114,NaN,9000,Reg
...,...,...,...
1813,NaN,3072,Reg
1866,NaN,11000,Reg
1941,NaN,7851,Reg
1947,NaN,12800,Reg


In [38]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna()
             )&(train['lot_area']<10000
               ),'lot_frontage'
         ].median()

60.0

In [39]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].isnull()
             )&(train['lot_area']<10000
               ),'lot_frontage'
         ] = 60

In [40]:
test.loc[(test['lot_shape']=='Reg'
           )&(test['lot_frontage'].isnull()
             )&(test['lot_area']<10000
               ),'lot_frontage'
         ] = 60

Replacement of null lot_frontage for 10_000<lot_area <20_000, with median value of lot_frontage for 10_000<lot_area <20_000

In [41]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna()
             )&(train['lot_area']>=10000
               )&(train['lot_area']<20000),'lot_frontage'
         ].median()

80.0

In [42]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].isnull()
             )&(train['lot_area']>=10000
               )&(train['lot_area']<20000),'lot_frontage'
         ]=80

In [43]:
test.loc[(test['lot_shape']=='Reg'
           )&(test['lot_frontage'].isnull()
             )&(test['lot_area']>=10000
               )&(test['lot_area']<20000),'lot_frontage'
         ]=80

Replacement of null lot_frontage for 20_000<lot_area <30_000, with median value of lot_frontage for 20_000<lot_area <30_000

In [44]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna()
             )&(train['lot_area']>=20000
               )&(train['lot_area']<30000),'lot_frontage'
         ].median()

100.0

In [45]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].isnull()
             )&(train['lot_area']>=20000
               )&(train['lot_area']<30000),'lot_frontage'
         ] = 100

In [46]:
test.loc[(test['lot_shape']=='Reg'
           )&(test['lot_frontage'].isnull()
             )&(test['lot_area']>=20000
               )&(test['lot_area']<30000),'lot_frontage'
         ] = 100

Replacement of null lot_frontage for 30_000<lot_area <40_000, with mean value of lot_frontage for 30_000<lot_area <40_000

In [47]:
train.loc[(train['lot_shape']=='Reg'
           )&(train['lot_frontage'].notna()
             )&(train['lot_area']>=30000
               )&(train['lot_area']<40000),'lot_frontage'
         ].mean()

112.5

In [48]:
train.loc[(train['lot_shape']=='Reg')&(train['lot_frontage'].isnull()
          )&(train['lot_area']>=30000)&(train['lot_area']<40000
           ),'lot_frontage'
         ] =112.5

In [49]:
test.loc[(test['lot_shape']=='Reg')&(test['lot_frontage'].isnull()
          )&(test['lot_area']>=30000)&(test['lot_area']<40000
           ),'lot_frontage'
         ] =112.5

Replacement of null lot_frontage for lot_area>=40_000, with mean value of lot_frontage for lot_area>=40_000

In [50]:
train.loc[(train['lot_shape']=='Reg')&(train['lot_frontage'].notna() )&(train['lot_area']>=40000
           ),'lot_frontage'
         ].median()

200.0

In [51]:
train.loc[(train['lot_shape']=='Reg')&(train['lot_frontage'].isnull())&(train['lot_area']>=40000
               ),'lot_frontage'
         ] = 200

In [52]:
test.loc[(test['lot_shape']=='Reg')&(test['lot_frontage'].isnull())&(test['lot_area']>=40000
               ),'lot_frontage'
         ] = 200

Check null value presence in lot_shape = reg

In [53]:
train.loc[(train['lot_shape']=='Reg')&(train['lot_frontage'].isnull()
             ),['lot_frontage','lot_area']]

,lot_frontage,lot_area


##### lot_shape = IR1

In [54]:
train.loc[(train['lot_shape']=='IR1')&(train['lot_frontage'].notna()),['lot_shape','lot_area','lot_frontage']
         ].sort_values(by=['lot_area'],ascending=False)

,lot_shape,lot_area,lot_frontage
1854,IR1,70761,62.0
1964,IR1,51974,52.0
1571,IR1,50271,68.0
616,IR1,47007,123.0
441,IR1,41600,195.0
...,...,...,...
1855,IR1,3784,32.0
156,IR1,3701,30.0
95,IR1,3523,80.0
847,IR1,3435,37.0


Since there is no proportionalty between lot_area and lot_frontage for lot_shape = IR1, nan values will be replaced by median value

In [55]:
train.loc[(train['lot_shape']=='IR1')&(train['lot_frontage'].notna()
             ),['lot_frontage']].median()

lot_frontage    73.0
dtype: float64

Filling up nan values with median value

In [56]:
train.loc[(train['lot_shape'] == 'IR1') & (train['lot_frontage'].isnull()
              ),'lot_frontage'] =73

In [57]:
test.loc[(test['lot_shape'] == 'IR1') & (test['lot_frontage'].isnull()
              ),'lot_frontage'] =73

##### lot_shape = IR2

In [58]:
train.loc[(train['lot_shape']=='IR2')&(train['lot_frontage'].notna()),['lot_shape','lot_area','lot_frontage']
         ].sort_values(by=['lot_area'],ascending=False)

,lot_shape,lot_area,lot_frontage
823,IR2,53504,47.0
1542,IR2,46589,52.0
1335,IR2,27650,313.0
1035,IR2,22950,90.0
886,IR2,21872,57.0
70,IR2,17485,44.0
1792,IR2,17227,73.0
800,IR2,17169,59.0
1976,IR2,16770,58.0
1753,IR2,16561,70.0


Since there is no proportionality between lot_area and lot_frontage, for lot_shape = IR2, nan values will be reaplaced by median values

In [59]:
train.loc[(train['lot_shape']=='IR2')&(train['lot_frontage'].notna()
            ),['lot_frontage']].median() #check for median values

lot_frontage    51.0
dtype: float64

In [60]:
len(train.loc[(train['lot_shape']=='IR2')&(train['lot_frontage'].isnull()
                ),'lot_frontage'])

19

Replacing null valuse with median value of 51

In [61]:
train.loc[(train['lot_shape']=='IR2')&(train['lot_frontage'].isnull()
            ),'lot_frontage'] = 51

In [62]:
test.loc[(test['lot_shape']=='IR2')&(test['lot_frontage'].isnull()
            ),'lot_frontage'] = 51

##### lot_shape = IR3

In [63]:
train.loc[(train['lot_shape']=='IR3')&(train['lot_frontage'].notna()),['lot_shape','lot_area','lot_frontage']
         ].sort_values(by=['lot_area'],ascending=False)

,lot_shape,lot_area,lot_frontage
960,IR3,63887,313.0
1701,IR3,33120,90.0
1155,IR3,21930,60.0
1006,IR3,16226,66.0
2023,IR3,12304,135.0
487,IR3,5232,30.0


for lotshape = IR3, aside for the outlier, of 135, there is an increasing lot_frontage length for increasing lot area
will replace lot_frontage below 10000 with 30, and replace 10000 to 50000 with 66

In [64]:
train.loc[(train['lot_shape']=='IR3')&(train['lot_frontage'].isnull()),['lot_shape','lot_area','lot_frontage']
         ].sort_values(by=['lot_area'],ascending=False)

,lot_shape,lot_area,lot_frontage
91,IR3,15295,NaN
1735,IR3,12155,NaN
941,IR3,9771,NaN


Replacing lot area less than 10000 with 30

In [65]:
train.loc[(train['lot_shape']=='IR3')&(train['lot_frontage'].isnull())&(train['lot_area']<10000)
          ,'lot_frontage']=30

In [66]:
test.loc[(test['lot_shape']=='IR3')&(test['lot_frontage'].isnull())&(test['lot_area']<10000)
          ,'lot_frontage']=30

Replacing lot area more than 10000 and less than 50000 with 66

In [67]:
train.loc[(train['lot_shape']=='IR3')&(train['lot_frontage'].isnull()
            )&
          (train['lot_area']>=10000) & (train['lot_area']<50000),'lot_frontage'] = 66

In [68]:
test.loc[(test['lot_shape']=='IR3')&(test['lot_frontage'].isnull()
            )&
          (test['lot_area']>=10000) & (test['lot_area']<50000),'lot_frontage'] = 66

Check if replacement is done

In [69]:
train['lot_frontage'].isnull().sum()

0

<font size = 3 color = "green">post project better solution for easier referencing.</font>

In [70]:
train.groupby(['neighborhood', 'lot_shape']).agg({'lot_area': 'median', 'lot_frontage': 'median'})

lot_area  lot_frontage
neighborhood lot_shape                        
Blmngtn      Reg          3189.0          53.0
Blueste      IR1          3907.0          35.0
             Reg          1697.0          24.0
BrDale       Reg          1680.0          21.0
BrkSide      IR1          7111.0          60.0
...                          ...           ...
Timber       IR1         11506.0          73.0
             Reg         11446.5          85.0
Veenker      IR1         15003.0          73.0
             IR2         11064.0          38.0
             Reg         12940.0          87.5

[75 rows x 2 columns]

### garage_yr_blt, ,garage_finish, garage_qual , garage_cond and garage_type

These are the relationships for garage_yr_blt,garage_finish, garage_qual , garage_cond and garage_type
1) garage_yr_blt vs garage_finish, if garage_yr_blt is not nan, garage_finish nan valuse cannot be NA, since there is a garage<br>
2) for garage_con, garage_finish, garage_qual, null Values cannot be NA if there is a garage present in garage_finish

Check unique values of garage_yr_blt

In [71]:
np.sort(train['garage_yr_blt'].unique())

array([1895., 1896., 1900., 1910., 1914., 1915., 1916., 1917., 1918.,
       1919., 1920., 1921., 1922., 1923., 1924., 1925., 1926., 1927.,
       1928., 1929., 1930., 1931., 1932., 1933., 1934., 1935., 1936.,
       1937., 1938., 1939., 1940., 1941., 1942., 1945., 1946., 1947.,
       1948., 1949., 1950., 1951., 1952., 1953., 1954., 1955., 1956.,
       1957., 1958., 1959., 1960., 1961., 1962., 1963., 1964., 1965.,
       1966., 1967., 1968., 1969., 1970., 1971., 1972., 1973., 1974.,
       1975., 1976., 1977., 1978., 1979., 1980., 1981., 1982., 1983.,
       1984., 1985., 1986., 1987., 1988., 1989., 1990., 1991., 1992.,
       1993., 1994., 1995., 1996., 1997., 1998., 1999., 2000., 2001.,
       2002., 2003., 2004., 2005., 2006., 2007., 2008., 2009., 2010.,
       2207.,   nan])

Check how many rows of garage_yr_blt = 2207

In [72]:
train.loc[train['garage_yr_blt']==2207,:]

,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,utilities,lot_config,...,3ssn_porch,screen_porch,pool_area,pool_qc,fence,misc_feature,misc_val,mo_sold,yr_sold,saleprice
1699,20,RL,68.0,8298,Pave,NA,IR1,HLS,AllPub,Inside,...,0,0,0,NA,NA,NA,0,9,2007,267300


there is a 2207 year for garage_yr_blt, it will be dropped 

In [73]:
train = train.drop(1699).reset_index(drop=True)

check rows where garage_finish is null, and garate_yr_blt is not null

In [74]:
train.loc[(train['garage_yr_blt'].notna())&(train['garage_finish'].isnull()
            ),['garage_yr_blt','garage_finish']]

,garage_yr_blt,garage_finish


since all null values of garage_yrblt and garage_finish is due to no garage, garage_cond, garage_cond,garage_type should be all null for this row 

In [75]:
train.loc[(train['garage_yr_blt'].isnull())&(train['garage_finish'].isnull()
            ),['garage_yr_blt','garage_finish','garage_cond','garage_qual','garage_type']
         ].isnull().sum()

garage_yr_blt    114
garage_finish    114
garage_cond      114
garage_qual      114
garage_type      113
dtype: int64

check row where garage_yr_blt and garage_finish is null but garage_type is not null

In [76]:
train.loc[(train['garage_yr_blt'].isnull())&(train['garage_finish'].isnull())
          &(train['garage_type'].notna()),['garage_yr_blt','garage_finish','garage_type']]

,garage_yr_blt,garage_finish,garage_type
1711,NaN,NaN,Detchd


Since row 1711 has a garage_type but no garage_yr_blt and garage_finish, row 1711 will be dropped from the dataframe

In [77]:
train = train.drop(1711).reset_index(drop=True)

Now that the remaining null values of the garage series are all due to no garage, the null values will be replaced by no garage equivalent

In [78]:
values = {'garage_yr_blt': 0,
          'garage_finish': 'NA',
          'garage_qual': 'NA',
          'garage_type': 'NA',
          'garage_cond': 'NA'
         }

In [79]:
train.fillna(value = values, inplace = True)

In [80]:
test.fillna(value = values, inplace = True)

Check updated null values row for train 

In [81]:
train.isnull().sum().sort_values(ascending=False).head(15)

bsmt_exposure     58
bsmtfin_type_2    56
bsmt_qual         55
bsmt_cond         55
bsmtfin_type_1    55
mas_vnr_type      21
mas_vnr_area      21
bsmt_full_bath     2
bsmt_half_bath     2
bsmtfin_sf_1       1
bsmtfin_sf_2       1
bsmt_unf_sf        1
total_bsmt_sf      1
gr_liv_area        0
kitchen_abvgr      0
dtype: int64

### bsmt_unf_sf, total_bsmt_sf

bsmt_unf_sf is total_bsmt_sf minus(bsmtfin_sf_1 + bsmtfin_sf_2)

In [82]:
train.loc[train['bsmt_unf_sf'].isnull(),
          ['bsmtfin_sf_1','bsmtfin_sf_2', 'total_bsmt_sf','bsmt_qual','bsmt_cond','bsmtfin_type_1','bsmtfin_sf_1',
          'bsmt_half_bath', 'bsmt_full_bath']]

,bsmtfin_sf_1,bsmtfin_sf_2,total_bsmt_sf,bsmt_qual,bsmt_cond,bsmtfin_type_1,bsmtfin_sf_1,bsmt_half_bath,bsmt_full_bath
1327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Since there is no indicator on whether there is a basement for row 1327, in order to not fillna wrongly or introduce unverified features, row 1327 will be dropped to avoid errorneous imputing based on assumption

In [83]:
train=train.drop(1327).reset_index(drop=True)

### bsmt_exposure

Since there is no relation to bsmt_exposure for other features, null values for bsmt_exposure will be filled as 0

In [84]:
train['bsmt_exposure'].fillna('NA', inplace=True)

In [85]:
test['bsmt_exposure'].fillna('NA', inplace=True)

### bsmtfin_type_1

bsmtfin_type_1 is the rating of basement finished area, if the bsmtfin_sf_1(type 1 finished square feet) is more than 0, bsmt_type_1 null values cannot be NA(no basement).  If bsmt_unf_sf is not zero, bsmtfin_type1 cannot be Unf(unfinished)

In [86]:
train.loc[train['bsmtfin_type_1'].isnull(),['bsmtfin_type_1', 'bsmtfin_sf_1','bsmt_unf_sf']]

,bsmtfin_type_1,bsmtfin_sf_1,bsmt_unf_sf
12,NaN,0.0,0.0
93,NaN,0.0,0.0
114,NaN,0.0,0.0
146,NaN,0.0,0.0
183,NaN,0.0,0.0
240,NaN,0.0,0.0
249,NaN,0.0,0.0
256,NaN,0.0,0.0
390,NaN,0.0,0.0
437,NaN,0.0,0.0


check for basement condition for row with null values of both bsmtfin_type_1 and bsmt_sf_1

In [87]:
train.loc[(train['bsmtfin_type_1'].isnull())&(train['bsmtfin_sf_1'].isnull()),['bsmtfin_type_1', 'bsmtfin_sf_1','bsmt_cond']]

,bsmtfin_type_1,bsmtfin_sf_1,bsmt_cond


none of the bsmtfin_type_1 null values row are within properties with basement, these null values will be replaced by NA

In [88]:
train['bsmtfin_type_1'].fillna('NA', inplace=True)

In [89]:
test['bsmtfin_type_1'].fillna('NA', inplace=True)

### bsmtfin_type_2

bsmtfin_type_2 is the rating of basement finished area, if the bsmtfin_sf_2(type 2 finished square feet) is more than 0, bsmt_type_2 null values cannot be NA(no basement). If bsmt_unf_sf is not zero, bsmtfin_type1 cannot be Unf(unfinished)

In [90]:
train.loc[train['bsmtfin_type_2'].isnull(),['bsmtfin_type_2', 'bsmtfin_sf_2','bsmt_unf_sf']]

,bsmtfin_type_2,bsmtfin_sf_2,bsmt_unf_sf
12,NaN,0.0,0.0
93,NaN,0.0,0.0
114,NaN,0.0,0.0
146,NaN,0.0,0.0
183,NaN,0.0,0.0
240,NaN,0.0,0.0
249,NaN,0.0,0.0
256,NaN,0.0,0.0
390,NaN,0.0,0.0
437,NaN,0.0,0.0


There is a row where bsmt_type_2 is null but bsmt_fin_sf_2 is not null. 

In [91]:
train.loc[train['bsmtfin_sf_2'] == 479,['bsmtfin_type_2', 'bsmtfin_sf_2']]

,bsmtfin_type_2,bsmtfin_sf_2
1147,NaN,479.0


finding out mode for basmtfin_type_2, for bsmtfin_sf_2 rows where there is no null values

In [92]:
train.loc[(train['bsmtfin_type_2']!='Unf')&(train['bsmtfin_type_2'].notna())&(train['bsmtfin_type_2'].notna()) ,'bsmtfin_type_2'].mode()

0    Rec
dtype: object

Assigning Rec to null value of bsmt_type_2 for row 1147

In [93]:
train.loc[train['bsmtfin_sf_2'] == 479,'bsmtfin_type_2']='Unf'

Assingning NA(no basement for type2) for remaining nan values

In [94]:
train['bsmtfin_type_2'].fillna('NA',inplace = True)

In [95]:
test['bsmtfin_type_2'].fillna('NA',inplace = True)

### bsmt_cond

null values of bsmt_cond cannot be NA(no basement) if bsmtfin_type_1 is not NA

In [96]:
train.loc[(train['bsmt_cond'].isnull()) & (train['bsmtfin_type_1']!= 'NA'),['bsmt_cond','bsmtfin_type_1']]

,bsmt_cond,bsmtfin_type_1


In [97]:
train.loc[train['bsmt_cond'].isnull(),['bsmt_cond','bsmtfin_type_1']]

,bsmt_cond,bsmtfin_type_1
12,NaN,NA
93,NaN,NA
114,NaN,NA
146,NaN,NA
183,NaN,NA
240,NaN,NA
249,NaN,NA
256,NaN,NA
390,NaN,NA
437,NaN,NA


Since all nan values of bsmt_cond fall under 'NA'(no basement of bsmtfin_type_1), all nan values of bsmt_cond will be replaced as 'NA'

In [98]:
train['bsmt_cond'].fillna('NA', inplace=True)

In [99]:
test['bsmt_cond'].fillna('NA', inplace=True)

### bsmt_qual

In [100]:
train.loc[train['bsmt_qual'].isnull(),['bsmtfin_sf_1','bsmt_qual']]

,bsmtfin_sf_1,bsmt_qual
12,0.0,NaN
93,0.0,NaN
114,0.0,NaN
146,0.0,NaN
183,0.0,NaN
240,0.0,NaN
249,0.0,NaN
256,0.0,NaN
390,0.0,NaN
437,0.0,NaN


since all values of bsmtfin_sf_1 is 0 for nan values of bsmt_qual, nan values will be replaced by NA(no basement)

In [101]:
train['bsmt_qual'].fillna('NA', inplace=True)

In [102]:
test['bsmt_qual'].fillna('NA', inplace=True)

### mas_vnr_type, mas_vnr_area

mas_vnr_type is related to mas_vnr_area

In [103]:
train.loc[train['mas_vnr_type'].isnull(),['mas_vnr_type','mas_vnr_area']]

,mas_vnr_type,mas_vnr_area
22,NaN,NaN
41,NaN,NaN
86,NaN,NaN
212,NaN,NaN
276,NaN,NaN
338,NaN,NaN
431,NaN,NaN
451,NaN,NaN
591,NaN,NaN
844,NaN,NaN


since both mas_vnr_type and mas_vnr_area has null values in the same row, and there are no other indicators for mas features

In [104]:
train['mas_vnr_type'].fillna('None', inplace=True)
train['mas_vnr_area'].fillna(0, inplace=True)

In [105]:
test['mas_vnr_type'].fillna('None', inplace=True)
test['mas_vnr_area'].fillna(0, inplace=True)

### bsmt_full_bath

In [106]:
train.loc[(train['bsmt_full_bath'].isnull())& (train['bsmt_half_bath'].isnull()),['bsmt_full_bath', 'bsmt_half_bath', 'bsmt_qual']]

,bsmt_full_bath,bsmt_half_bath,bsmt_qual
616,NaN,NaN,NA


since the nan values of bsmt_half_bath and bsmt_full_bath falls under no basement. nan values will be changed to 0

In [107]:
train['bsmt_full_bath'].fillna(0,inplace=True)

In [108]:
train['bsmt_half_bath'].fillna(0,inplace=True)

In [109]:
train.isnull().sum().sum()

0

### check for test

check for any null values left in test

In [110]:
test.isnull().sum().sum()

1

In [111]:
test[test.columns[test.isnull().any()]].isnull().sum().sort_values(ascending=False)

electrical    1
dtype: int64

In [112]:
test.loc[test['electrical'].isnull(),['electrical','utilities']]

,electrical,utilities
635,NaN,AllPub


Since it is supposed to have all utilities, reference will be made from other houses with AllPub for electrical data imputing

In [113]:
train.loc[train['utilities']=='AllPub','electrical'].unique()

array(['SBrkr', 'FuseF', 'FuseA', 'FuseP', 'Mix'], dtype=object)

In [114]:
train.loc[train['utilities']=='AllPub','electrical'].mode()

0    SBrkr
dtype: object

since there are 5 different type of unique values for when there is AllPub, i will impute it using the mode value

In [115]:
test.loc[test['electrical'].isnull(),'electrical'] = 'SBrkr'

In [116]:
test.loc[635,'electrical']

'SBrkr'

In [117]:
test.isnull().sum().sum()

0

In [124]:
train.isnull().sum().sum()

0

## converting cleaned test and train dataset to csv file

In [121]:
train.to_csv('./datasets/cleaned_train.csv', na_rep='NA',index=False)

In [119]:
# test.to_csv('./datasets/cleaned_test.csv',index=False)

In [126]:
test.to_csv('./datasets/cleaned_test2.csv',na_rep='NA',index=False)